## Base Funcs

In [36]:
import pandas as pd
import jieba
import itertools
from tqdm.auto import tqdm

ERROR: Could not find a version that satisfies the requirement tqdm.auto (from versions: none)
ERROR: No matching distribution found for tqdm.auto
You should consider upgrading via the '/Users/gwihwango/.pyenv/versions/3.7.3/envs/py_selenium/bin/python -m pip install --upgrade pip' command.


In [19]:
# !pip install jieba
# !pip install emoji
import jieba
def segmentate_sentence(sent):
    """
    word segmentation with jieba
    ref : https://github.com/fxsjy/jieba
    Args :
        sent(Str) : a sentence without segmentation, e.g "Iloveyou"
    Output :
        sent(str) : segmentated sentence, e.g "I love you"
    """
    # sent = clean(sent)
    seg_list = jieba.lcut(sent, cut_all=False)
    
    seg_list = [i for i in seg_list if i!=" "]
    return seg_list
    
def clean(x):
    """clean tweets
    :param: a tweet (str)
    :return a preprocessed tweet (str)
    """
    import emoji
    from collections import Counter
    import re
    emojis = ''.join(emoji.UNICODE_EMOJI.keys())
    pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
    username_pattern = re.compile(
        r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)")
    url_pattern = re.compile(
        r"[-a-zA-Z0-9@:%_\+.~#?&//=]{2,256}\.[a-z]{2,4}\b(\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?")

    chinese_pattern=re.compile(r"[^\u4e00-\u9fa5]+")
#     x = username_pattern.sub('', x)
#     x = url_pattern.sub('', x)
    x = chinese_pattern.sub('', x)
#     x = re.sub(r'[0-9]+', '', x)
#     table = x.maketrans({ '/': ' ',  '#': ' ', '@':' ', ':':' '})
#     x = x.translate(table)
    x = re.sub(pattern, '', x)
#     x = re.sub(r"[^a-zA-Z0-9]","",x) # 특수문자 제거
    x = x.strip()
    return x

## Jieba

In [ ]:
# !pip uninstall googletrans==4.0.0-rc1

###Make it as word and save on table###
###table_col : mymeaning, MT_meaning, count, position
# jieba.add_word('石墨烯')
# jieba.add_word('办主任')
# jieba.del_word('自定义词')

df = pd.read_csv("train_data/6_5.csv")

jieba.load_userdict("./userdict.txt") # load customized dictionary

sentences = [None]*df.shape[0]
for row in range(df.shape[0]) :
    sentences[row] = segmentate_sentence(df['English title'][row])
sentences

In [2]:
test_sent = (
"李小福是创新办主任也是云计算方面的专家; 什么是八一双鹿\n"
"例如我输入一个带“韩玉赏鉴”的标题，在自定义词库中也增加了此词为N类\n"
"「台中」正確應該不會被切開。mac上可分出「石墨烯」；此時又可以分出來凱特琳了。"
)
words = jieba.cut(test_sent)
print('/'.join(words))

print("="*40)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jt/jg8f8t5d5l94gkq96yn3k41h0000gn/T/jieba.cache
Loading model cost 0.743 seconds.
Prefix dict has been built successfully.


李小福/是/创新/办/主任/也/是/云/计算/方面/的/专家/;/ /什么/是/八/一双/鹿/
/例如/我/输入/一个/带/“/韩玉/赏鉴/”/的/标题/，/在/自定义词/库中/也/增加/了/此/词为/N/类/
/「/台/中/」/正確/應該/不會/被/切開/。/mac/上/可/分出/「/石墨/烯/」/；/此時/又/可以/分出/來凱/特琳/了/。


## Data - One hot encoding

One hot encoding including position infos.

meaning, category, position info

1. save on csv - with checking if there is this word
2. add meaning
3. add position meaning

words - word - meaning - cate - 
1
2
3
4

In [66]:
####Working On DATA####
import pandas as pd

df_paths = ["6_4", "6_5", "6_6", "6_7"]
cols = ['word', 'meaning', 'cate', 'position']



In [68]:

bag_of_words = set()

def add_words(raw_paths, all_dicts=set()) :
    """
    add new data with existing data
    input :
        raw_paths : list of path in train_data folder
        all_dicts : existing data, if not given, it initalize the dataset
    output :
        set of words
    """
    for path in tqdm(raw_paths) :
        print(len(list(all_dicts)))
        df = pd.read_csv(f"train_data/{path}.csv")
        sentences = [None]*df.shape[0]

        for row in range(df.shape[0]) :
            sentences[row] = segmentate_sentence(df['English title'][row])

        flatten = list(itertools.chain(*sentences))
        new_words = set(set(flatten)).difference(set(all_dicts))
        all_dicts = list(all_dicts) + list(new_words)
    print(f"final_length : {len(list(all_dicts))}")
    return all_dicts

bag_of_words = add_words(df_paths)
len(list(bag_of_words))
###make a word list


100%|██████████| 4/4 [00:00<00:00, 33.34it/s]

0
492
833
1063
final_length : 1270


1270

In [53]:
def write_dict(bag_of_words) :
    """
    write the dictinory with xlsx file 
    category :
        brand(0), main_feature(1-7), kinds(8), unimportant_feature(9)
    input : 
        list of words
    output : 
        None
    """
    # !pip install googletrans==3.1.0a0
    # ref) https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group
    # ref) https://pypi.org/project/googletrans/
    from googletrans import Translator
    gmt = Translator()
    new_list = [[i,'',0,0] for i in bag_of_words]
    for i in tqdm(new_list) :
        i[1] = gmt.translate(i[0], dest='ko').text
    
    ######Save as Excel#####
    df = pd.DataFrame(new_list)
    writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='welcome', index=False)
    writer.save()

write_dict(bag_of_words)

100%|██████████| 1270/1270 [05:57<00:00,  3.55it/s]


In [62]:
## Make a function of read xlsx to 
df = pd.read_excel('test.xlsx')
set(df.loc[:,0])

{'内',
 '年',
 '五条',
 '泡泡袖',
 '风琴',
 '袋盖',
 '磨毛',
 '小花',
 'Polo',
 'CIVICERA',
 '到',
 '春秋',
 '身裙',
 'X',
 '双面',
 '通勤',
 '翻领',
 '150',
 '粉色',
 '两件套',
 '小衫',
 '菠萝',
 '中国',
 '襟',
 '原单',
 '雪中飞',
 '女外',
 '在',
 '孔雀蓝',
 '情侣装',
 '珠',
 '流苏',
 'lulu',
 '你家',
 '港味',
 '知',
 '绒',
 '风雪',
 '基础',
 '紧身',
 '坠',
 '茶',
 '搭配',
 '酵素',
 '男士',
 '许大晴',
 '臀',
 '中长款',
 '法式',
 '小鹅',
 '露背',
 '分割',
 '花瓣',
 '丁香',
 'APEA2022',
 '古高街',
 'CALVINLUO',
 '春秋季',
 '感夏',
 '2toyoung',
 '设计师',
 '猫咪',
 '捡漏',
 '【',
 '粗花呢',
 '被',
 '200',
 'DIDDI',
 '清仓',
 '短袖',
 '女小香',
 '三角裤',
 '背带',
 '一周',
 '街头时尚',
 '网红',
 '衣服',
 '果哥',
 '皮草',
 '搭外',
 '一城',
 '型',
 '钉珠',
 '礼服',
 '~',
 '21',
 '收腰',
 '码',
 '貂',
 '性感',
 '穿',
 '收腹',
 '纯色',
 '肌理',
 '品牌',
 '妖精',
 '199',
 '上',
 '可',
 '卡其色',
 '&',
 '青色',
 '宋慧乔',
 '火龙果',
 '百搭中',
 '摆',
 'CHANG',
 '摩妮',
 '瘦长',
 '金大支',
 '草莓',
 '元',
 '拉飞姆',
 '港味百',
 '芭比',
 '字阔',
 '对称',
 'LE',
 '仙',
 '风格',
 '老板',
 '垫肩',
 '婚礼',
 '铆钉',
 '小老虎',
 'Eat',
 'cuijia',
 '菇',
 '云朵',
 '22FS1343',
 '软',
 '系带',
 '透气',
 '甜辣',
 '果果',
 'FT',
 '瞌

In [ ]:
##새로 추가된 값 데이터 생성해서 기존 자료에 추가.

In [50]:
# !pip install googletrans==3.1.0a0
# ref) https://stackoverflow.com/questions/52455774/googletrans-stopped-working-with-error-nonetype-object-has-no-attribute-group
# ref) https://pypi.org/project/googletrans/
from googletrans import Translator
gmt = Translator()
print(gmt.translate('裤卫裤', dest='ko').text)

운동복


In [ ]:
###Opn other files and save on this table too###
###No overlap